## X. Two-stage XGBoost (frequency × severity)

In my main XGBoost notebook I already approximate claim counts directly with a
single Poisson model (`count:poisson`), which implicitly combines:

- the probability of having at least one claim, and  
- the expected number of claims, conditional on having a claim.

In this notebook I implement a **clean two-stage (hurdle) decomposition** of
the same idea:

1. A **frequency model** (`XGBClassifier`) that estimates  
   \\( p(\text{ClaimNb} > 0 \mid X) \\),
2. A **severity-on-positives model** (`XGBRegressor` with Poisson objective)
   fitted only on policies with \\( \text{ClaimNb} > 0 \\),
3. The final expected count as  
   \\[
   \hat{\mu}(X) = \hat{p}(\text{ClaimNb} > 0 \mid X)\times
                   \hat{\mathbb{E}[\text{ClaimNb} \mid \text{ClaimNb} > 0, X]}.
   \\]

This two-stage implementation is **slightly cleaner and more modular** than the
earlier hurdle-style experiment in the main notebook (separate tuning for the
classifier and for the positive-claim regressor, with optional tail-weighting).

However, the **out-of-sample results are very similar** to the single
Poisson XGBoost model in terms of Poisson deviance and RMSE.  
In other words, the two-stage approach mainly serves as a **validation check**
and an alternative modelling view (frequency × severity), rather than a clear
performance improvement for this dataset.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    mean_poisson_deviance,
    root_mean_squared_error,
    roc_auc_score,
    average_precision_score,
    make_scorer,
)

from xgboost import XGBClassifier, XGBRegressor

warnings.filterwarnings("ignore")

sns.set(style="whitegrid")


target_col = "ClaimNb"
exposure_col = "Exposure"
id_col = "IDpol"

feature_cols = [c for c in df.columns if c not in [target_col, exposure_col, id_col]]

X = df[feature_cols]
y = df[target_col]
exposure = df[exposure_col]

y_has_claim = (y > 0).astype(int)

X_train, X_test, y_train, y_test, exp_train, exp_test, y_train_has, y_test_has = train_test_split(
    X,
    y,
    exposure,
    y_has_claim,
    test_size=0.2,
    random_state=42,
    stratify=y_has_claim,
)

X_train.shape, X_test.shape, y_train_has.mean(), y_test_has.mean()

((542410, 13),
 (135603, 13),
 np.float64(0.05023506203794178),
 np.float64(0.050234876809510116))

In [2]:
cat_features = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
num_features = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("Categorical features:", cat_features)
print("Numeric features:", num_features)

preprocess = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
    ]
)

poisson_scorer = make_scorer(mean_poisson_deviance, greater_is_better=False)

skf = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42,
)

Categorical features: ['Area', 'VehBrand_lumped', 'VehGas', 'Region_lumped', 'DrivAgeBand', 'VehAgeBand', 'BonusMalusBand']
Numeric features: ['VehPower', 'VehAge', 'DrivAge', 'BonusMalus', 'Density', 'logDensity']


In [3]:
xgb_clf = XGBClassifier(
    objective="binary:logistic",
    tree_method="hist",
    device="cuda",
    eval_metric="logloss",
    random_state=42,
)

clf_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", xgb_clf),
])

param_clf = {
    "model__n_estimators":     [300, 500, 700],
    "model__max_depth":        [3, 4, 5],
    "model__learning_rate":    [0.03, 0.05, 0.07],
    "model__subsample":        [0.8, 1.0],
    "model__colsample_bytree": [0.8, 1.0],
    "model__min_child_weight": [1, 3, 5],
}

clf_search = RandomizedSearchCV(
    estimator=clf_pipe,
    param_distributions=param_clf,
    n_iter=25,
    scoring="roc_auc",
    cv=skf,
    verbose=2,
    n_jobs=-1,
    random_state=42,
)

clf_search.fit(
    X_train,
    y_train_has,
    model__sample_weight=exp_train,
)

print("Best frequency (classifier) params:", clf_search.best_params_)
print("Best CV AUC:", clf_search.best_score_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best frequency (classifier) params: {'model__subsample': 0.8, 'model__n_estimators': 700, 'model__min_child_weight': 3, 'model__max_depth': 5, 'model__learning_rate': 0.03, 'model__colsample_bytree': 0.8}
Best CV AUC: 0.6575288431969822


In [4]:
best_clf = clf_search.best_estimator_

p_test = best_clf.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test_has, p_test)
ap = average_precision_score(y_test_has, p_test)

print(f"Frequency model – Test AUC: {auc:.4f}")
print(f"Frequency model – Test AP:  {ap:.4f}")

Frequency model – Test AUC: 0.6588
Frequency model – Test AP:  0.1116


E:\ml forecasting\.venv\Lib\site-packages\xgboost\core.py:774: UserWarning: [21:02:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


In [5]:
mask_pos = y_train > 0

X_train_pos = X_train[mask_pos]
y_train_pos = y_train[mask_pos]
exp_train_pos = exp_train[mask_pos]

print("Train policies:", len(y_train))
print("Train policies with ClaimNb > 0:", len(y_train_pos))
print("Mean ClaimNb among positives:", y_train_pos.mean())

Train policies: 542410
Train policies with ClaimNb > 0: 27248
Mean ClaimNb among positives: 1.059380504991192


In [6]:
xgb_sev = XGBRegressor(
    objective="count:poisson",
    tree_method="hist",
    device="cuda",
    eval_metric="poisson-nloglik",
    random_state=42,
)

sev_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", xgb_sev),
])

param_sev = {
    "model__n_estimators":      [300, 500, 700],
    "model__max_depth":         [3, 4, 5, 6],
    "model__learning_rate":     [0.03, 0.05, 0.07],
    "model__subsample":         [0.8, 0.9, 1.0],
    "model__colsample_bytree":  [0.7, 0.9, 1.0],
    "model__min_child_weight":  [1, 3, 5],
    "model__gamma":             [0.0, 0.1, 0.3],
    "model__reg_lambda":        [1.0, 5.0, 10.0],
    "model__reg_alpha":         [0.0, 0.5, 1.0],
}

tail_weight = np.where(y_train_pos >= 3, 3.0, 1.0)
sev_sample_weight = exp_train_pos * tail_weight

sev_search = RandomizedSearchCV(
    estimator=sev_pipe,
    param_distributions=param_sev,
    n_iter=30,
    scoring=poisson_scorer,
    cv=5,
    verbose=2,
    n_jobs=-1,
    random_state=123,
)

sev_search.fit(
    X_train_pos,
    y_train_pos,
    model__sample_weight=sev_sample_weight,
)

print("Best severity params:", sev_search.best_params_)
print("Best severity CV Poisson dev:", sev_search.best_score_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best severity params: {'model__subsample': 0.9, 'model__reg_lambda': 10.0, 'model__reg_alpha': 0.0, 'model__n_estimators': 700, 'model__min_child_weight': 3, 'model__max_depth': 6, 'model__learning_rate': 0.05, 'model__gamma': 0.3, 'model__colsample_bytree': 0.9}
Best severity CV Poisson dev: -0.04877772033214569


In [7]:
best_sev = sev_search.best_estimator_

# 1) P(has_claim | X)
p_hat_test = best_clf.predict_proba(X_test)[:, 1]

# 2) E[ClaimNb | ClaimNb>0, X]
sev_hat_test = best_sev.predict(X_test)
sev_hat_test = np.clip(sev_hat_test, 1e-6, None)

# 3) Final prediction
y_hat_two_stage = p_hat_test * sev_hat_test

two_stage_poisson_dev = mean_poisson_deviance(y_test, y_hat_two_stage)
two_stage_rmse = root_mean_squared_error(y_test, y_hat_two_stage)

print(f"Two-stage XGB Test Poisson deviance: {two_stage_poisson_dev:.4f}")
print(f"Two-stage XGB Test RMSE:             {two_stage_rmse:.4f}")

Two-stage XGB Test Poisson deviance: 0.3064
Two-stage XGB Test RMSE:             0.2385
